# CSV Export

This notebook is for visualizing the CSV exports from source files to Django.

In [1]:
import pandas as pd

In [2]:
cols = [
    'ExhibitionID',
    'ExhibitionNumber',
    'ExhibitionTitle',
    'ConstituentURL', 
    'FirstName',
    'MiddleName',
    'LastName',
    'Suffix',
    'ExhibitionURL',
    'ExhibitionRole',
    'DisplayName',
]

exh = pd.read_csv(
    '~/data1/moma/exhibitions/MoMAExhibitions1929to1989.csv', 
    usecols=cols,
    dtype={
        'ExhibitionID': 'Int64',
    },
    converters={
        'ExhibitionTitle': str,
        'FirstName': str,
        'LastName': str,
        'MiddleName': str,
        'Suffix': str,
    },
    encoding="iso8859-1",
)

## The complete list of columns

['ExhibitionID',
 'ExhibitionNumber',
 'ExhibitionTitle',
 'ExhibitionCitationDate',
 'ExhibitionBeginDate',
 'ExhibitionEndDate',
 'ExhibitionSortOrder',
 'ExhibitionURL',
 'ExhibitionRole',
 'ExhibitionRoleinPressRelease',
 'ConstituentID',
 'ConstituentType',
 'DisplayName',
 'AlphaSort',
 'FirstName',
 'MiddleName',
 'LastName',
 'Suffix',
 'Institution',
 'Nationality',
 'ConstituentBeginDate',
 'ConstituentEndDate',
 'ArtistBio',
 'Gender',
 'VIAFID',
 'WikidataID',
 'ULANID',
 'ConstituentURL']
 

## Filter for artists

The CSV contains one role for each artist in a given exhibition. So let's filter for only artists.

In [3]:
artists = exh.loc[exh['ExhibitionRole'] == 'Artist']
artists = artists.loc[artists['ExhibitionTitle'] != "No#"]

In [4]:
artists

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionURL,ExhibitionRole,DisplayName,FirstName,MiddleName,LastName,Suffix,ConstituentURL
1,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Paul Cézanne,Paul,,Cézanne,,moma.org/artists/1053
2,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Paul Gauguin,Paul,,Gauguin,,moma.org/artists/2098
3,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Vincent van Gogh,Vincent,,van Gogh,,moma.org/artists/2206
4,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Georges-Pierre Seurat,Georges-Pierre,,Seurat,,moma.org/artists/5358
5,2724,2,Paintings by 19 Living Americans,moma.org/calendar/exhibitions/1912,Artist,Charles Burchfield,Charles,,Burchfield,,moma.org/artists/870
...,...,...,...,...,...,...,...,...,...,...,...
34552,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Akihiko Tsukamoto,Akihiko,,Tsukamoto,,moma.org/artists/5959
34553,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Aijiro Wakita,Aijiro,,Wakita,,moma.org/artists/6215
34554,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Yoji Yamamoto,Yoji,,Yamamoto,,moma.org/artists/6486
34555,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Ryuichi Yamashiro,Ryuichi,,Yamashiro,,moma.org/artists/6487


## Add a column for the Gensim token

Since the Gensim tokenizer trimmed trailing 'e's and otherwise altered artist names, it would be cool to have a column for the token, so that Django can translate those names when interacting with the model.

In [5]:
from gensim.parsing.preprocessing import preprocess_string

In [6]:
def format_name(names):
    "Join a name with an underscore"
    # process all the names at once.
    return ["".join(preprocess_string(n)) for n in names]

In [7]:
artists_tokenized = artists.assign(
    token=lambda x: format_name(x.DisplayName)
)

In [8]:
artists_tokenized

,ExhibitionID,ExhibitionNumber,ExhibitionTitle,ExhibitionURL,ExhibitionRole,DisplayName,FirstName,MiddleName,LastName,Suffix,ConstituentURL,token
1,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Paul Cézanne,Paul,,Cézanne,,moma.org/artists/1053,paulcézann
2,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Paul Gauguin,Paul,,Gauguin,,moma.org/artists/2098,paulgauguin
3,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Vincent van Gogh,Vincent,,van Gogh,,moma.org/artists/2206,vincentvangogh
4,2557,1,"Cézanne, Gauguin, Seurat, Van Gogh",moma.org/calendar/exhibitions/1767,Artist,Georges-Pierre Seurat,Georges-Pierre,,Seurat,,moma.org/artists/5358,georgpierrseurat
5,2724,2,Paintings by 19 Living Americans,moma.org/calendar/exhibitions/1912,Artist,Charles Burchfield,Charles,,Burchfield,,moma.org/artists/870,charlburchfield
...,...,...,...,...,...,...,...,...,...,...,...,...
34552,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Akihiko Tsukamoto,Akihiko,,Tsukamoto,,moma.org/artists/5959,akihikotsukamoto
34553,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Aijiro Wakita,Aijiro,,Wakita,,moma.org/artists/6215,aijirowakita
34554,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Yoji Yamamoto,Yoji,,Yamamoto,,moma.org/artists/6486,yojiyamamoto
34555,288,1536,Recent Japanese Posters from the Collection,moma.org/calendar/exhibitions/1739,Artist,Ryuichi Yamashiro,Ryuichi,,Yamashiro,,moma.org/artists/6487,ryuichiyamashiro


## Export time!

All right, now we have a mapping to the Gensim model's token. 

TODO:

This is good, but there is one thing that would make it better. This script exports _all_ the exhibitions, but only some of them are included in the model. Exhibitions above a threshold were not included.

```python
for en in exh_numbers:
        terms = Moma.exhibition_artists(en)
        # Don't calculate and output big lists
        if len(terms) <= 50:
            Moma.append_to_outfile(terms)
```


This is cool, but the dataframe isn't ready yet. We need to remove exhibitions that are above or equal to 50.

In [7]:
exh_numbers = artists_tokenized.ExhibitionNumber.unique()

drop_indices = []

for exn in exh_numbers:
    # Select a group of rows by ExhibitionNumber
    e = artists_tokenized.loc[artists_tokenized["ExhibitionNumber"] == str(exn)]
    # Create a list of rows to drop
    if len(e) >= 50:
        for i in e.index.tolist():
            drop_indices.append(i)

# Drop the oversize exhibitions
tokenized_filtered = artists_tokenized.drop(drop_indices)

In [8]:
# Export the CSV
tokenized_filtered.to_csv(
    index=False,
    path_or_buf='../data/artists_tokenized_filtered.csv',
)